In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train =pd.read_csv(r'../input/reduce-marketing-waste-hackerearth-ml-challenge/train.csv') 
test = pd.read_csv(r'../input/reduce-marketing-waste-hackerearth-ml-challenge/test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

* Its generally a good idea to combine both train and test data sets into one, perform feature engineering and then divide them later again. This saves the trouble of performing the same steps twice on test and train. Lets combine them into a dataframe ‘data’ with a ‘source’ column specifying where each observation belongs.

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.nunique()

##### dropping unique values features

In [ ]:
data.drop(['Geography','Lead_name','Contact_no','POC_name','Lead_POC_email'],axis=1,inplace=True)

### Data Cleaning & Feature engineering

In [ ]:
data.isnull().sum()

In [ ]:
data['Date_of_creation'] = pd.to_datetime(data['Date_of_creation'].values,format='%Y-%m-%d')

In [ ]:
data['Designation'].value_counts()

In [ ]:
data['Designation'] = data['Designation'].replace({'Chairman/CEO/President':'Chairman/CEO/President',
                                              'CEO/Chairman/President':'Chairman/CEO/President',
                                              'Chief Executive Officer':'CEO',
                                              'Vice President / GM (04-present) : VP Sales and Marketing (01-04)':'Vice President/GM'})

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='Designation', data = data)
plt.show()

In [ ]:
data['Fund_category'].value_counts()

In [ ]:
plt.figure(figsize=(14,7))
sns.countplot(x='Fund_category', data = data)
plt.show()

In [ ]:
data['Industry'].value_counts()

In [ ]:
data['Industry'].replace(np.nan,'Banks',inplace=True)

In [ ]:
data['Internal_rating'].value_counts()

In [ ]:
data['Internal_rating'].replace({-1.00:1.00,
                                82.34:4.00},inplace=True)

In [ ]:
plt.figure(figsize=(14,7))
sns.countplot(x='Internal_rating', data = data)
plt.show()

In [ ]:
data['Last_lead_update'].value_counts()

In [ ]:
data['Last_lead_update'].replace(np.nan,'No track',inplace=True)

In [ ]:
data['Location'].value_counts()

In [ ]:
data['Location'].replace(np.nan,'Aurangabad',inplace=True)

In [ ]:
data['Deal_value'] = data['Deal_value'].str.replace('$', '')
data['Weighted_amount'] = data['Weighted_amount'].str.replace('$', '')

In [ ]:
data['Weighted_amount'] = data['Weighted_amount'].astype(float)
data['Deal_value'] = data['Deal_value'].astype(float)

In [ ]:
data['Weighted_amount'].fillna(data['Weighted_amount'].mean(),inplace=True)
data['Deal_value'].fillna(data['Deal_value'].mean(),inplace=True)

In [ ]:
data['Resource'].value_counts()

In [ ]:
data['Resource'] = data['Resource'].map({'We have all the requirements':'Yes',
                                        'Cannot deliver':'No',
                                        'Not enough':'No',
                                        'Deliverable':'Yes'})

In [ ]:
data['Resource'].replace(np.nan,'Yes',inplace=True)

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='Resource', data = data)
plt.show()

In [ ]:
data.isnull().sum()

In [ ]:
data['Last_lead_update'].value_counts()

In [ ]:
data['Last_lead_update'].replace('?','No track',inplace=True)

In [ ]:
sns.boxplot(x = 'Success_probability',data = data)
plt.show()

* Checking if **Success_probability** is greater than 100 or smaller than 0,

In [ ]:
data.loc[data['Success_probability']>100]

In [ ]:
data.loc[data['Success_probability']<0]

* Looks like there is some values let's fix them.

In [ ]:
data['Success_probability'] = data['Success_probability'].apply(lambda x:100.0 if x>100 else x)
data['Success_probability'] = data['Success_probability'].apply(lambda x:0.0 if x<0 else x)

* Now let's convert data back into train and test data sets. Its generally a good idea to export both of these as modified data sets so that they can be re-used for multiple sessions. This can be achieved using following code:

In [ ]:
data.drop('Internal_POC',axis=1,inplace=True)
train_m = data.loc[data['source']=='train']
test_m = data.loc[data['source']=='test']

test_m.drop(['source','Success_probability',],axis=1,inplace=True)
train_m.drop('source',axis=1,inplace=True)

### Data Visualization

In [ ]:
df = pd.DataFrame(train_m['Success_probability'].values,index=train_m['Date_of_creation'],columns=['Success_proabability'])

In [ ]:
weekly=df.resample(rule='W').mean()
plt.figure(figsize=(20,5))
sns.lineplot(data=weekly, palette = ['green'])
plt.title('Success_probability Weekly Variation')
plt.ylabel('Success_Probability')
plt.show()

In [ ]:
month = df.resample(rule='M').mean()
plt.figure(figsize=(20,5))
sns.lineplot(data = month, palette = ['green'])
plt.title('Success_probability Monthly Variation')
plt.ylabel('Success_Probability')
plt.show()

In [ ]:
year = df.resample(rule='A').mean()
plt.figure(figsize=(20,5))
sns.lineplot(data = year, palette = ['green'])
plt.title('Success_probability Yearly Variation')
plt.ylabel('Success_Probability')
plt.show()

Not finished, I am currently working on it.